In [1]:
import requests
import nltk
import math
import re
import regex as re
import pandas as pd
import numpy as np
import statistics as stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import spacy
nlp = spacy.load("en_core_web_sm") # English Word Pipline In Spacy

from bs4 import BeautifulSoup 
from nltk import *
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nerd import ner

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nikolaiklassen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nikolaiklassen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Small fucntion for the World Bank Blog Page to extract the links to the Blog Posts.

def get_hrefs(page,class_name):
    page=requests.get(page)
    soup=BeautifulSoup(page.text,'html.parser')
    container=soup.find_all('div',{'class':class_name})
    
    lst= []
    for i in range(len(container)):
        lst.append(container[i].find_all('a')[0])
    
    links = re.findall('"/([^"]*)"', str(lst)) # REGEX to get the link
    hrefs = []
    for link in links: 
            hrefs.append('https://blogs.worldbank.org/'+link)
    
    return hrefs

In [5]:
url = 'https://blogs.worldbank.org/opendata'
links = get_hrefs(url,'field field--name-field-short-title field--type-string field--label-hidden field--item')

In [6]:
links

['https://blogs.worldbank.org/opendata/making-policy-more-inclusive-digital-age-lessons-those-cutting-edge',
 'https://blogs.worldbank.org/opendata/local-prices-global-economy-how-data-international-comparison-program-help-us-monitor',
 'https://blogs.worldbank.org/opendata/economic-impact-second-lockdown-uganda-results-seventh-round-high-frequency-phone-survey',
 'https://blogs.worldbank.org/opendata/how-data-can-accelerate-equality',
 'https://blogs.worldbank.org/opendata/commodity-prices-surged-february-pink-sheet',
 'https://blogs.worldbank.org/opendata/observation-external-debt-interest-payments-after-global-crises',
 'https://blogs.worldbank.org/opendata/local-prices-global-economy-how-data-international-comparison-program-help-us-monitor',
 'https://blogs.worldbank.org/opendata/size-asia-and-pacific-economy-based-purchasing-power-parities-results-international',
 'https://blogs.worldbank.org/opendata/demystifying-icp-purchasing-power-parity-calculations-using-python',
 'https://

In [77]:
def get_date(soup):
    dates = []
    try:
        data = soup.find('span',{'class':'views-field views-field-field-date'})
        content=data.find('time')
        dates.append(content['datetime'])
    except Exception:
        dates.append('')
        pass
    return dates[0] 

In [78]:
def get_title(soup):
    try:
        title=soup.find('h1',{'class':'field-content'}).contents
        return title[0]
    except Exception:
        return 'Unknown Title'
        pass

In [115]:
def get_contents(soup):
    try:
        parents_blacklist=['[document]','html','head',
                           'style','script','body',
                           'div','a','section','tr',
                           'td','label','ul','header',
                           'aside',]
        content=''
        text=soup.find_all(text=True)
        
        for t in text:
            if t.parent.name not in parents_blacklist and len(t) > 10:

                content=content+t+' '
        
        return content 

    except Exception:
        content_text.append('')
        pass

In [116]:
dates = []
title = []
content_text = ''
for link in links:
    page=requests.get(link)
    soup=BeautifulSoup(page.text,'html.parser')
    dates.append(get_date(soup))
    title.append(get_title(soup))
    content_text += get_contents(soup) # maybe not very elegant

In [233]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

data = word_tokenize(content_text)
data_lower = [dat.lower() for dat in data]

In [229]:
import pycountry as pc 
import operator

def get_countries(content_list):
    '''
    Hashmap makes it linear instead of O(N^3) 
    Returns desc. sorted key value pairs of counrtries mentioned in a list,
    '''
    country_list = [name.name.lower() for name in list(pc.countries)]
    country_dict = {i:0 for i in country_list }  

    for i in content_list:
        try:
            country_dict[i] +=1
        except:
            continue 
    return dict(sorted(country_dict.items(), key=operator.itemgetter(1),reverse=True))

In [232]:
countries = get_countries(data_lower)
countries

{'china': 35,
 'ghana': 14,
 'india': 11,
 'mexico': 7,
 'uganda': 7,
 'brazil': 6,
 'estonia': 5,
 'indonesia': 5,
 'egypt': 4,
 'finland': 4,
 'malaysia': 4,
 'singapore': 4,
 'kenya': 3,
 'cambodia': 3,
 'qatar': 3,
 'sudan': 3,
 'thailand': 3,
 'australia': 2,
 'djibouti': 2,
 'fiji': 2,
 'maldives': 2,
 'nigeria': 2,
 'nepal': 2,
 'pakistan': 2,
 'peru': 2,
 'philippines': 2,
 'rwanda': 2,
 'angola': 1,
 'bangladesh': 1,
 'bahrain': 1,
 'belarus': 1,
 'bhutan': 1,
 'canada': 1,
 'denmark': 1,
 'ethiopia': 1,
 'iraq': 1,
 'jordan': 1,
 'kazakhstan': 1,
 'kuwait': 1,
 'luxembourg': 1,
 'morocco': 1,
 'myanmar': 1,
 'mongolia': 1,
 'oman': 1,
 'aruba': 0,
 'afghanistan': 0,
 'anguilla': 0,
 'åland islands': 0,
 'albania': 0,
 'andorra': 0,
 'united arab emirates': 0,
 'argentina': 0,
 'armenia': 0,
 'american samoa': 0,
 'antarctica': 0,
 'french southern territories': 0,
 'antigua and barbuda': 0,
 'austria': 0,
 'azerbaijan': 0,
 'burundi': 0,
 'belgium': 0,
 'benin': 0,
 'bonaire,

In [239]:
text_file = open("Content.txt", "w")
text_file.write(content_text)
text_file.close()

In [245]:
# making sentences tokens from the content text
sent_text = nltk.sent_tokenize(content_text)

tokenizer = WordPunctTokenizer()

data_tok = [tokenizer.tokenize(x.lower()) for x in data]


In [246]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [255]:
# taking the word vectors from the model

words = sorted(model.key_to_index.keys(), 
               key=lambda word: model.key_to_index[word],
               reverse=True)

word_vectors = np.array([model.get_vector(word) for word in words])

In [256]:
from sklearn.decomposition import PCA

word_vectors_pca = PCA(2).fit_transform(word_vectors)

# Transforming the variables
from sklearn.preprocessing import StandardScaler
word_vectors_pca = StandardScaler().fit_transform(word_vectors_pca)


In [257]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [258]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)


Figure(id='1005', ...)